In [287]:
import pandas as pd
import os

# Nombre del archivo a leer
name_input = "registro.csv"

# Direccion del archivo a leer
path_file_input = os.path.join(os.getcwd(), name_input)

registro = pd.read_csv(path_file_input)


### Top 10 de palabras que se encuentran primero de todas las partidas.

In [297]:
# Saco columnas que no uso
partidas = registro[['ID', 'Timestamp', 'Evento', 'Estado', 'Respuesta']]

# Filtro buscando los registros de inicio y respuesta correcta
partidas = partidas[(partidas['Evento'] == 'inicio_partida') | (partidas['Estado'] == 'ok')]

# Agrupo por partida
partidas = partidas.groupby('ID')

# Le resto al tiempo de la primera respuesta correcta [1] el tiempo de inicio [0]
# para saber cuanto se tardo en responder la primera correcta
respuestas = []
tiempos = []
for uuid, dato in partidas:
    if len(dato['Timestamp']) > 1:
        respuestas.append(dato['Respuesta'].iloc[1])
        tiempos.append(dato['Timestamp'].iloc[1] - dato['Timestamp'].iloc[0])

tiempos_respuestas = pd.DataFrame(data={'Respuesta': respuestas, 'Tiempo': tiempos})
tiempos_respuestas = tiempos_respuestas.sort_values('Tiempo')
tiempos_respuestas.head(10)


,Respuesta,Tiempo
10,Christoffer Aasbak,0.960487
9,Tinakula (Solomon Is.),1.029762
0,Lago Salitroso o Sucio,1.230529
5,Lago Onelli,1.313998
11,Iliwerung (Indonesia),1.441111
3,"Chichon, El (Mexico)",1.999027
8,Lago Traful,2.128344
7,Adrián Rocheira,4.159255
1,Lago Strobel,5.612406
4,Vesuvius (Italy),5.617003


### Duración total de la partida (desde que el usuarie empieza a jugar hasta que presiona salir o pasan las N rondas definidas para el nivel de dificultad actual).

In [298]:
aux = registro[['ID', 'Timestamp']]

eventos_partidas = aux.groupby('ID')

tiempo_de_partidas = []
index_uuid = []
for uuid, dato in eventos_partidas:
    index_uuid.append(uuid)
    tiempo_de_partidas.append(dato.iloc[len(dato) - 1][1] - dato.iloc[0][1])

duracion_total_partidas = pd.DataFrame(data={'Tiempo': tiempo_de_partidas}, index=index_uuid)
duracion_total_partidas


,Tiempo
03a4cb9e-b643-4506-85a8-d547e2815dd0,9.136203
0403cb52-5b98-470e-8868-8c2c4972f0fe,14.157984
0c65f958-7273-495b-a55e-3cc3788b38e3,86.694448
103c5824-34c3-4bde-a6f6-0ea5cf749df7,6.697662
16092f51-1607-467b-966d-d8ecf3a489d0,8.063008
190c0160-8b97-4c7b-9556-641e68c9334f,2.459541
20930f46-980e-48b0-9a9a-a32d8dfbc614,13.818453
24c97e43-23a7-4153-b9ba-545b4e4b7620,1.075764
2e79aec8-5220-4ecd-8fc5-9ef51852d89b,72.723168
4c56a963-5a64-4d50-887b-980cceb535b8,12.505075


### Tiempo de la respuesta exitosa más rápida.

In [299]:
# Saco columnas que no uso
aux = registro[['Timestamp', 'Evento', 'Estado']]

# Elimino las filas que tengan el evento fin
eventos_sin_fin = aux[aux['Evento'] != 'fin']

tiempo_respuestas_correctas = []
for i in range(len(eventos_sin_fin)):
    if eventos_sin_fin.iloc[i][2] == 'ok':
        tiempo_respuestas_correctas.append(eventos_sin_fin.iloc[i][0] - eventos_sin_fin.iloc[i-1][0])

respuesta_mas_rapida = (pd.DataFrame(data={'Tiempo': tiempo_respuestas_correctas})
                          .sort_values('Tiempo')
                          .head(1)
                        )
respuesta_mas_rapida = respuesta_mas_rapida.iloc[0][0]

print(f'La respuesta mas rápida fue en {respuesta_mas_rapida:0.2f} segundos.')


La respuesta mas rápida fue en 0.16 segundos.


### Promedio de tiempo de respuesta para respuestas exitosas.


In [300]:
# Usa el tiempo_respuestas_correctas calculado en "Tiempo de la respuesta exitosa más rápida."

aux = registro[['Evento']]
cantidad_respuestas = aux[aux['Evento'] == 'Intento'].count()[0]

tiempo_total = pd.DataFrame(data={'Tiempo': tiempo_respuestas_correctas}).sum()[0]
tiempo_promedio_respuesta = tiempo_total / cantidad_respuestas

print(f'El tiempo promedio de respuesta correcta es de {tiempo_promedio_respuesta:0.2f} segundos.')


El tiempo promedio de respuesta correcta es de 0.28 segundos.


### Promedio de tiempo de partida general.

In [301]:
# Usa el tiempo_de_partidas calculado en "Duración total de la partida"

tiempo_promedio_partida = pd.DataFrame(data={'Tiempo': tiempo_de_partidas})
tiempo_promedio_partida = tiempo_promedio_partida.mean()[0]

print(f'El tiempo promedio de partida es de {tiempo_promedio_partida:0.2f} segundos.')


El tiempo promedio de partida es de 26.07 segundos.


### Porcentaje de aciertos por usuarie.

In [303]:
intentos = registro[['Usuario', 'Evento', 'Estado']]
intentos = intentos[intentos['Evento'] == 'Intento']

total_intentos = intentos.groupby(['Usuario']).size()
cantidad_bien = intentos[intentos['Estado'] == 'ok'].groupby(['Usuario']).size()

data = pd.DataFrame(data={'Bien': cantidad_bien, 'Total': total_intentos})
data['Porcentaje'] = (cantidad_bien / total_intentos * 100)
data


,Bien,Total,Porcentaje
Usuario,,,
Agustin,11,41,26.829268
Franco,11,52,21.153846
Sofia,12,50,24.000000


### Cantidad de tarjetas para las que el usuarie no dió respuesta (timeout)

In [296]:
cant_intentos = registro[registro['Evento'] == 'Intento']
cant_timeouts = cant_intentos[cant_intentos['Estado'] == 'timeout'].groupby(['Usuario']).size()
data = pd.DataFrame(data = {'Veces que no dio respuesta':cant_timeouts})

data

,Veces que no dio respuesta
Usuario,
Agustin,1
Franco,2
Sofia,1


### Cantidad de tarjetas en las que el usuarie dió una respuesta errónea.

In [295]:
cant_errores = cant_intentos[cant_intentos['Estado'] == 'error'].groupby(['Usuario']).size()
data = pd.DataFrame(data = {'Veces se equivoco':cant_errores})

data

,Veces se equivoco
Usuario,
Agustin,25
Franco,34
Sofia,34
